# Import and Publish Projects
This Jupyter notebook uses the SAS Event Stream Processing Studio Rest API and the SAS Event Stream Manager Rest API to import, publish, and run projects in a Kubernetes cluster.


Prerequisite: This example assumes that there is an *xml_projects* folder at same level as this Jupyter notebook, which contains XML files that you want to import into SAS Event Stream Processing Studio and publish to SAS Event Stream Manager.

This Jupyter notebook uses the project XML files in the `xml_projects` folder and performs the following tasks:
1. Check whether the projects have already been imported to SAS Event Stream Processing Studio.
2. If a project has been previously imported, then import it using the next version number. Otherwise, import the project as version 1.
3. Make the projects public so that they are visible to all users.
4. Publish the projects from SAS Event Stream Processing Studio to SAS Event Stream Manager.
5. Synchronize the projects.
6. Create a SAS Event Stream Manager deployment whose type is "Cluster".
7. Run the projects in the Kubernetes cluster. This action creates and starts an ESP server for each project.

   
**Important**: You must run the [Imports and Global Variables](#imports) cell before executing anything else in this notebook. Also ensure that you are authenticated by running the [Get Access Token](#authentication) cell.


## Imports and Global Variables <a id='imports'></a>
Run this cell before any of the others as it imports packages and sets variables that are used throughout this notebook.

In [ ]:
import requests
import xml.etree.ElementTree as ET
import os
import json
import time
import sys
from urllib3.exceptions import InsecureRequestWarning


def bootstrap_server_and_credentials():
    global server, username, password, chosen_deployment_name
    server =  "https://your_server"
    username = "your_username"
    password = "your_password"
    chosen_deployment_name = "test"
    if len(sys.argv) > 3:
        server = sys.argv[1]
        username = sys.argv[2]
        password = sys.argv[3]

    print('Server: ' + server)
    print('Username: ' + username)
    print('Password: ' + password, flush=True)
    print('Name of SAS Event Stream Manager Deployment: ' + chosen_deployment_name, flush=True)

# Suppress ssl warnings caused by verify=False
requests.packages.urllib3.disable_warnings(category=InsecureRequestWarning)
bootstrap_server_and_credentials()

# Get Access Token <a id='authentication'></a>

In [ ]:
def get_access_token():
    body = {'grant_type': 'password', 'username': username, 'password': password}
    headers = {'Content-Type': 'application/x-www-form-urlencoded', 'Authorization': 'Basic c2FzLmVjOg=='}
    access_token_response = requests.post(server + '/SASLogon/oauth/token', data=body, headers=headers,
                                          verify=False)
    return access_token_response.json()["access_token"]

access_token = get_access_token()
print(access_token)
headers = {"Content-Type": "application/json", "Authorization": "Bearer " + access_token}

# Function to Get ESP Projects

In [5]:
def get_projects():
    projects = []
    get_projects_response = requests.get(
        server + '/SASEventStreamProcessingStudio/esp-project', headers=headers, verify=False)
    if get_projects_response.status_code == 200:
        projects = get_projects_response.json()["items"]
    return projects

# Function to Import a Project to SAS Event Stream Processing Studio

In [6]:
def import_project_to_studio(project_body):
    project_id = ""
    import_project_response = requests.post(server + '/SASEventStreamProcessingStudio/esp-project',
                                            data=json.dumps(project_body), headers=headers, verify=False)
    if import_project_response.status_code == 200:
        project_id = import_project_response.json()["flowId"]
        print('success, project_id=' + project_id)
    return project_id

# Function to Get the Next Version Number of a Project

In [7]:
def get_next_project_version(project_id):
    version = 2
    get_next_version_response = requests.get(
        server + '/SASEventStreamProcessingStudio/project-versions/projects/' + project_id + '/nextVersion',
        headers=headers, verify=False)
    if get_next_version_response.status_code == 200:
        version = get_next_version_response.json()["major"]
    else:
        print('Failed to get next version')
        print(get_next_version_response)
    return version

# Function to Make a Project Public
By default, a project is private and hidden from other users.

In [8]:
def make_project_public(project_id):
    requests.patch(server + '/SASEventStreamProcessingStudio/esp-project/'
                   + project_id + '/authorization?private=false',
                   headers={'Authorization': 'Bearer ' + access_token}, verify=False)

# Function to Create the Expected Project Model to be Published

In [9]:
def create_publish_project_body(project_body, version):
    project_body["name"] = project_body["friendlyName"]
    project_body["description"] = ""
    project_body["friendlyName"] = None
    project_body["majorVersion"] = str(version)
    project_body["minorVersion"] = "0"
    project_body["version"] = str(version) + '.0'
    project_body["versionNotes"] = "notes"
    project_body["uploadedBy"] = username
    project_body["modifiedBy"] = username
    epoch_time = int(time.time())
    project_body["uploaded"] = epoch_time
    project_body["modified"] = epoch_time + 10
    project_body["isDeployable"] = False

# Function to Publish a Project

In [10]:
def publish_project(project_id, project_body, version):
    folder_id = ""
    create_publish_project_body(project_body, version)
    publish_project_response = requests.post(
        server + '/SASEventStreamProcessingStudio/project-versions/projects/' + project_id,
        data=json.dumps(project_body), headers=headers, verify=False)

    if publish_project_response.status_code == 200:
        folder_id = publish_project_response.json()["folderId"]
    else:
        print('PUBLISH FAILED')
        print(publish_project_response)
        print('Version:' + str(version))
    return folder_id

# Function to Synchronize Projects from SAS Event Stream Processing Studio to SAS Event Stream Manager

In [11]:
def synchronize_project_for_ESM(folder_id):
    success = False
    synchronize_project_response = requests.post(
        server + '/SASEventStreamProcessingStudio/project-versions/projects/synchronize/' + folder_id,
        data=folder_id, headers=headers, verify=False)

    if synchronize_project_response.status_code == 200:
        success = True
    return success

# Function to Get Deployment Details Needed to Start Kubernetes Cluster in SAS Event Stream Manager

In [12]:
def get_deployment_details():
    success = True
    deployment_id = ''
    deployment_name = ''
    projects_running_on_deployment = []
    deployments_response = requests.get(server + "/SASEventStreamManager/deployment?noDetails=false",
                                        headers=headers, verify=False)

    if deployments_response.status_code != 200:
        print("Could not find any deployments", deployments_response.text)
        success = False

    # Here we try to start clusters against the hard-coded deployment name
    # if it does not exist, it will spin off cluster against 1st cluster
    deployment_items = deployments_response.json()["items"]
    if len(deployment_items) > 0:
        deployment_id = deployment_items[0]["uuid"]
        deployment_name = deployment_items[0]["label"]
        projects_running_on_deployment = get_project_names_from_deployment(deployment_items[0])
        
        for deployment in deployment_items:
            if deployment["type"] == "cluster" and deployment["label"] == chosen_deployment_name:
                deployment_id = deployment["uuid"]
                deployment_name = deployment["label"]
                projects_running_on_deployment = get_project_names_from_deployment(deployment)
    else:
        print("Could not find any deployments", deployments_response.text)
        success = False

    return success, deployment_id, deployment_name, projects_running_on_deployment

def get_project_names_from_deployment(deployment):
    project_names = []
    for server in deployment["servers"]:
        project_names = project_names + list(map(lambda project: project["name"], server["projects"]))
    return project_names

# Function to Start Kubernetes Cluster in SAS Event Stream Manager

In [13]:
def start_k8s_cluster(k8s_project_body):
    success, deployment_id, deployment_name, projects_running_on_deployment = get_deployment_details()

    if k8s_project_body["name"] in projects_running_on_deployment:
        print("Project " + k8s_project_body["name"] + " is already running on deployment " + deployment_name +".")
    elif success:
        # Here are the deployment settings hard-coded for all projects
        k8s_deployment_settings = {
            "persistentVolumeClaim": "sas-event-stream-processing-studio-app",
            "requestsMemory": "1Gi",
            "requestsCpu": 1,
            "requestsGpu": "0",
            "limitsMemory": "1Gi",
            "limitsCpu": 1,
            "limitsGpu": "0",
            "minReplicas": 1,
            "maxReplicas": 1,
            "useLoadBalancer": False,
            "loadBalancingPolicy": "none",
            "averageUtilization": "50",
            "loadBalancerTargetsList": []
        }
        k8s_cluster_body = {
            "distinguisher": deployment_name,
            "esmDeploymentId": deployment_id,
            "gpuReliant": False,
            "loadOnly": False,
            "project": k8s_project_body,
            "settings": k8s_deployment_settings
        }

        response = requests.post(server + "/SASEventStreamManager/server/cluster",
                                 data=json.dumps(k8s_cluster_body), headers=headers, verify=False)
        if response.status_code != 200:
            print("error creating cluster", response.text)
        else:
            print("Project " + k8s_project_body["name"], " successfully started K8s pod in deployment: " + deployment_name)

# Create a SAS Event Stream Manager Deployment in Which Projects Can Run

In [3]:
def create_ESM_cluster_deployment():
    deployment_alread_exists_error = 'A deployment named "' + chosen_deployment_name + '" already exists'
    deployment_body = {
        "name": chosen_deployment_name,
        "type": "cluster"
    }
    response = requests.post(server + '/SASEventStreamManager/deployment',
                                            data=json.dumps(deployment_body), headers=headers, verify=False)
    if deployment_alread_exists_error in response.text:
        print ("Deployment " + chosen_deployment_name + " already exists, we can proceed to start the projects")
    elif response.status_code != 201:
        print("error creating ESM cluster deployment", response.text)

create_ESM_cluster_deployment()

# Loop Through All Project XML Files in the `xml_projects` Folder and Perform the Following Steps
1. Check whether the projects have already been imported to SAS Event Stream Processing Studio.
2. If a project has been previously imported, then import it using the next version number. Otherwise, import the project as version 1.
3. Make the projects public so that they are visible to all users.
4. Publish the projects from SAS Event Stream Processing Studio to SAS Event Stream Manager.
5. Synchronize the projects.
6. Create a SAS Event Stream Manager deployment whose type is "Cluster".
7. Run the projects in the Kubernetes cluster. This action creates and starts an ESP server for each project.

In [ ]:
def import_and_publish_xml_files():
    current_dir = os.getcwd() + "/xml_projects"
    projects = get_projects()
    for file_name in os.listdir(current_dir):
        print()
        if not file_name.endswith('.xml'): continue
        version = 1
        project_id = None
        xml_file_path = os.path.join(current_dir, file_name)
        project_name = get_project_name_from_xml(xml_file_path)
        data = open(xml_file_path, "r").read()
        project_body = {'friendlyName': project_name, 'xml': data}

        if projects:
            project_that_matches_xml_name = None
            for project in projects:
                if project['friendlyName'] == project_name:
                    project_that_matches_xml_name = project
                    break

            if project_that_matches_xml_name:
                print(project_name + ' is already imported')
                project_id = project_that_matches_xml_name['flowId']
                version = get_next_project_version(project_id)
                print('Creating new project version: ' + str(version))

        if not bool(project_id):
            print('Importing ' + project_name)
            project_id = import_project_to_studio(project_body)
            make_project_public(project_id)

        folder_id = publish_project(project_id, project_body, version)

        synchronize_project_for_ESM(folder_id)

        if project_id:
            print(file_name + ' successfully published', flush=True)
        else:
            print(file_name + ' failed to publish', flush=True)

        k8s_project_body = {"id": project_id, "name": project_name, "version": version}
        start_k8s_cluster(k8s_project_body)

def get_project_name_from_xml(xml_file_path):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()
    project_name = root.attrib['name']
    return project_name



import_and_publish_xml_files()
